In [ ]:
!pip install gdown
!pip install tqdm
!pip install ultralytics


import gdown         
import os
import shutil
import zipfile
from google.colab import files 
import random
import math


import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from collections import deque

import torch
import torch.nn as nn
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, WeightedRandomSampler, DataLoader, SubsetRandomSampler
from torchvision import transforms
import torch.optim as optim
from sklearn.model_selection import train_test_split

import cv2
from tqdm import tqdm 
from PIL import Image
from IPython.display import display, FileLink, Markdown
from collections import defaultdict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torch.cuda.amp as amp


url_videos = "https://drive.google.com/drive/folders/1nO_javaDEk0yFcglOX7WoqRNj_MtCEDd?usp=drive_link"
gdown.download_folder(url_videos, quiet=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:00


['/content/train/reid_checkpoints/phase1_reid_hybrid_ENDZONE_epoch_20.pth',
 '/content/train/reid_checkpoints/phase1_reid_hybrid_SIDELINE_epoch_20.pth',
 '/content/train/best_dualview_96.pth',
 '/content/train/best_siamese_matcher_endzone.pth',
 '/content/train/best_siamese_matcher.pth',
 '/content/train/best_weights_players_detection.pt',
 '/content/train/best.pt',
 '/content/train/ckpt.t7',
 '/content/train/csv_helmets.zip',
 '/content/train/embeddings.zip',
 '/content/train/helmet_dataset.yaml',
 '/content/train/helmets_extracted.zip',
 '/content/train/reid_gt_metadata_ENDZONE.csv',
 '/content/train/reid_gt_metadata_SIDELINE.csv',
 '/content/train/reid_gt_metadata.csv',
 '/content/train/train_player_tracking_frames.csv',
 '/content/train/video_directions.csv',
 '/content/train/Videos_dataset.zip']

In [ ]:

original_path = "train"
new_base = "datasets"
new_path = os.path.join(new_base, "Dataset")

os.makedirs(new_base, exist_ok=True)

if not os.path.exists(new_path):
    shutil.move(original_path, new_path)
    print(f" Folder '{original_path}' moved in '{new_path}'")
dataset_path = "datasets/Dataset"

# extract zip files and remove them
def extract_and_remove_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)
    print(f"File zip {zip_path} extracted and removed.")

zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
for zip_file in zip_files:
    zip_path = os.path.join(dataset_path, zip_file)
    extract_and_remove_zip(zip_path, dataset_path)

✅ Cartella 'train' spostata in 'datasets/Dataset'
File zip datasets/Dataset/helmets_extracted.zip estratto e rimosso.
File zip datasets/Dataset/Videos_dataset.zip estratto e rimosso.
File zip datasets/Dataset/csv_helmets.zip estratto e rimosso.
File zip datasets/Dataset/embeddings.zip estratto e rimosso.


#**DATASET PREPROCESSING**

In [ ]:
FRAME_W = 1280      
FRAME_H = 720       
IMPACT_LABELS_PATH = "/content/datasets/Dataset/Videos_dataset/train_labels.csv"

# SELECT VIEW ('Endzone' or 'Sideline')
TARGET_VIEW = 'Endzone'

# VIDEOS FOR TESTING
TEST_VIDEOS = [
    "57906_000718_Endzone.mp4", "57906_000718_Sideline.mp4",
    "57995_000109_Endzone.mp4", "57995_000109_Sideline.mp4",
    "58102_002798_Endzone.mp4", "58102_002798_Sideline.mp4",
]



class VideoAugmentationPipeline:
    def __init__(self, p_occlusion=0.4, p_switch=0.3):
        self.p_occlusion = p_occlusion
        self.p_switch = p_switch

        self.stats = {
            'total_sequences': 0,
            'clean': 0,
            'occlusion_applied': 0,
            'id_switch_applied': 0,
            'both_applied': 0
        }

    def __call__(self, images):
        self.stats['total_sequences'] += 1
        has_occlusion = False
        has_switch = False

        if len(images) > 4 and random.random() < self.p_occlusion:
            num_frames_to_drop = random.randint(1, 3)
            indices_to_drop = random.sample(range(len(images)), num_frames_to_drop)

            w, h = images[0].size
            black_frame = Image.new('RGB', (w, h), (0, 0, 0))

            for idx in indices_to_drop:
                images[idx] = black_frame

            has_occlusion = True

        if len(images) > 6 and random.random() < self.p_switch:
            mode = random.choice(['start', 'end'])
            block_size = random.randint(4, 6)

            w, h = images[0].size
            black_frame = Image.new('RGB', (w, h), (0, 0, 0))

            if mode == 'end':
                for i in range(len(images) - block_size, len(images)):
                    images[i] = black_frame
            else:
                for i in range(block_size):
                    images[i] = black_frame

            has_switch = True

        if has_occlusion and has_switch:
            self.stats['both_applied'] += 1
        elif has_occlusion:
            self.stats['occlusion_applied'] += 1
        elif has_switch:
            self.stats['id_switch_applied'] += 1
        else:
            self.stats['clean'] += 1

        return images

    def get_and_reset_stats(self):
        if self.stats['total_sequences'] == 0:
            return "No data processed."

        total = self.stats['total_sequences']
        report = (
            f" REPORT AUGMENTATION (on {total} sequences):\n"
            f"   - Cleared: {self.stats['clean']} ({self.stats['clean']/total:.1%})\n"
            f"   - Only Occlusion : {self.stats['occlusion_applied']} ({self.stats['occlusion_applied']/total:.1%})\n"
            f"   - Only ID Switch : {self.stats['id_switch_applied']} ({self.stats['id_switch_applied']/total:.1%})\n"
            f"   - Extreme: {self.stats['both_applied']} ({self.stats['both_applied']/total:.1%})"
        )

        # Reset
        self.stats = {k: 0 for k in self.stats}
        return report

def get_train_transforms():
    return T.Compose([
        T.RandomApply([T.Resize((48, 48)), T.Resize((64, 64))], p=0.2),
        T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))], p=0.3),
        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
        T.ToTensor(),
        T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
    ])

def get_val_transforms():
    return T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
    ])


def convert_to_tlbr(df):
    df['x1'] = df['left']
    df['y1'] = df['top']
    df['x2'] = df['left'] + df['width']
    df['y2'] = df['top'] + df['height']
    return df

def bbox_iou_raw(boxA, boxB):
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])
    interArea = max(0, x2 - x1) * max(0, y2 - y1)
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = float(areaA + areaB - interArea)
    return interArea / union if union > 0 else 0.0

def extract_sliding_subsequences(file_path, target_view):

    SAMPLE_LENGTH = 16
    FRAME_STRIDE = 4
    MAX_SPAN_ALLOWED = 25

    df = pd.read_csv(file_path)
    df = df[~df['video'].isin(TEST_VIDEOS)].copy() 

    df = df[df['view'] == target_view].copy() 
    df['impact'] = df['impact'].fillna(0)
    df['label'] = df['label'].astype(str)
    df = convert_to_tlbr(df)

    if 'label' in df.columns and 'player_id' not in df.columns:
        df.rename(columns={'label': 'player_id'}, inplace=True)

    all_subsequences = []
    track_groups = df.groupby(['gameKey', 'playID', 'view', 'player_id'])

    for (gameKey, playID, view, player_id), track_df_full in tqdm(track_groups, desc=f"Processing {target_view} Tracks"):
        track_df_full = track_df_full.sort_values('frame').reset_index(drop=True)
        frames = track_df_full['frame'].values

        if len(frames) < SAMPLE_LENGTH: continue

        for i in range(0, len(frames) - SAMPLE_LENGTH + 1, FRAME_STRIDE):
            sub_track_df = track_df_full.iloc[i : i + SAMPLE_LENGTH].copy()

            start_frame_val = sub_track_df['frame'].iloc[0]
            end_frame_val = sub_track_df['frame'].iloc[-1]
            if (end_frame_val - start_frame_val) > MAX_SPAN_ALLOWED: continue

            impact_vector = sub_track_df['impact'].values.astype(np.float32)
            is_impact_sequence = (np.sum(impact_vector) > 0.5)

            video_name = sub_track_df['video'].iloc[0]
            image_meta = []
            for _, row in sub_track_df.iterrows():
                image_meta.append({
                    'video_folder': video_name,
                    'file_name': f"{row['player_id']}_{row['frame']}.jpg",
                    'frame_idx': row['frame'],
                    'bbox': [row['left'], row['top'], row['width'], row['height']]
                })

            all_subsequences.append({
                'key': f"{gameKey}_{playID}_{view}_{player_id}_{start_frame_val}",
                'is_impact': is_impact_sequence,
                'data': image_meta,
                'targets': impact_vector
            })

    print(f"\n Extraction {target_view} completed. Total subtracks: {len(all_subsequences)}")
    return all_subsequences

data_samples = extract_sliding_subsequences(IMPACT_LABELS_PATH, TARGET_VIEW)

⚙️ Configurazione impostata per visuale: Endzone
🚫 Esclusione di 6 video di test dal dataset di training.

🔬 Inizio estrazione SOLO per Endzone...
   (Include fix NaN e Tolleranza Buchi Temporali)
   ✂️ Rimossi 56230 record appartenenti ai video di test.


Processing Endzone Tracks: 100%|██████████| 1168/1168 [02:14<00:00,  8.67it/s]


✅ Estrazione Endzone completata. Totale sottotracce: 101881


In [ ]:
class NFLImpactDataset(Dataset):
    def __init__(self, samples, root_dir, phase='train', transform=None, video_augmenter=None, sigma=2):
        self.samples = samples
        self.root_dir = root_dir
        self.phase = phase
        self.transform = transform
        self.video_augmenter = video_augmenter
        self.sigma = sigma

    def __len__(self):
        return len(self.samples)

    def generate_gaussian_target(self, impact_vector, length=16):
        
        impact_vector = np.nan_to_num(np.array(impact_vector), nan=0.0)
        impact_indices = np.where(impact_vector > 0.5)[0]
        target = np.zeros(length, dtype=np.float32)

        if len(impact_indices) == 0:
            return torch.tensor(target, dtype=torch.float32)

        x = np.arange(length)
        for idx in impact_indices:
            gauss = np.exp(-((x - idx)**2) / (2 * self.sigma**2))
            target = np.maximum(target, gauss)

        return torch.tensor(target, dtype=torch.float32)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        meta_data = sample['data']

        images = []

        for item in meta_data:
            video_folder = item['video_folder'].replace('.mp4', '')
            img_path = os.path.join(self.root_dir, video_folder, item['file_name'])

            try:
                img = Image.open(img_path).convert('RGB')
                if img.size != (64, 64):
                    img = img.resize((64, 64))
            except Exception:
                img = Image.new('RGB', (64, 64), (0, 0, 0))

            images.append(img)

        # VIDEO AUGMENTATION (Masking/Occlusions)
        if self.phase == 'train' and self.video_augmenter:
            images = self.video_augmenter(images)

        # IMAGE TRANSFORMS (Blur, Color, Tensor, Norm)
        processed_frames = []
        do_flip = (self.phase == 'train' and random.random() > 0.5)

        for img in images:
            if do_flip:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)

            if self.transform:
                img_t = self.transform(img)
            else:
                img_t = T.ToTensor()(img)

            processed_frames.append(img_t)

        video_tensor = torch.stack(processed_frames).permute(1, 0, 2, 3)

        raw_targets = sample['targets']
        smooth_targets = self.generate_gaussian_target(raw_targets)

        return video_tensor, smooth_targets

#**Model Definition**

In [ ]:
class NFLImpactModel(nn.Module):
    def __init__(self, num_frames=16, hidden_dim=256, dropout_prob=0.5):
        super(NFLImpactModel, self).__init__()

        weights = R2Plus1D_18_Weights.DEFAULT
        self.backbone = r2plus1d_18(weights=weights)

        in_features = self.backbone.fc.in_features

        self.backbone.fc = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Linear(in_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim), 
            nn.ReLU(),
            nn.Dropout(p=dropout_prob), 
            nn.Linear(hidden_dim, num_frames) 
        )

    def forward(self, x):
        features = self.backbone(x) # Output: (Batch, 512)

        out = self.classifier(features) # Output: (Batch, 16)

        return out

#**Training Phase**

In [ ]:
print(f"START SETUP TRAINING FOR: {TARGET_VIEW.upper()}")


all_video_names = list(set([s['data'][0]['video_folder'] for s in data_samples]))
print(f"Total Videos for training phase: {len(all_video_names)}")

train_videos, val_videos = train_test_split(all_video_names, test_size=0.2, random_state=42)

print(f"   -> Videos in Train: {len(train_videos)}")
print(f"   -> Videos in Val:   {len(val_videos)}")

train_samples_all = [s for s in data_samples if s['data'][0]['video_folder'] in train_videos]
val_samples_all = [s for s in data_samples if s['data'][0]['video_folder'] in val_videos]

train_pos = [s for s in train_samples_all if np.sum(s['targets']) > 0.5]
train_neg = [s for s in train_samples_all if np.sum(s['targets']) < 0.5]

val_pos = [s for s in val_samples_all if np.sum(s['targets']) > 0.5]
val_neg = [s for s in val_samples_all if np.sum(s['targets']) < 0.5]

print(f"   TRAIN -> Positives: {len(train_pos)} | Negatives: {len(train_neg)}")
print(f"   VAL   -> Positives: {len(val_pos)}   | Negatives: {len(val_neg)}")

train_samples_final = train_pos + train_neg
val_samples_final = val_pos + val_neg

train_dataset = NFLImpactDataset(
    train_samples_final,
    root_dir="/content/datasets/Dataset/helmets_extracted",
    phase='train',
    transform=get_train_transforms(),
    video_augmenter=VideoAugmentationPipeline(p_occlusion=0.4, p_switch=0.3)
)

val_dataset = NFLImpactDataset(
    val_samples_final,
    root_dir="/content/datasets/Dataset/helmets_extracted",
    phase='val',
    transform=get_val_transforms()
)

train_targets = [1 if np.sum(s['targets']) > 0.5 else 0 for s in train_samples_final]
class_counts = np.bincount(train_targets)
weights = 1.0 / np.maximum(class_counts, 1)
samples_weights = torch.DoubleTensor([weights[t] for t in train_targets])

train_sampler = WeightedRandomSampler(weights=samples_weights, num_samples=15000, replacement=True)

val_targets = np.array([1 if np.sum(s['targets']) > 0.5 else 0 for s in val_samples_final])
val_pos_idxs = np.where(val_targets == 1)[0]
val_neg_idxs = np.where(val_targets == 0)[0]
num_neg_val = min(len(val_neg_idxs), len(val_pos_idxs) * 3)
val_balanced_idxs = np.concatenate([val_pos_idxs, np.random.choice(val_neg_idxs, num_neg_val, replace=False)])
val_sampler = SubsetRandomSampler(val_balanced_idxs)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=2, pin_memory=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path_best_f1 = f"nfl_r2plus1d_{TARGET_VIEW}_BEST_F1.pth"
path_best_loss = f"nfl_r2plus1d_{TARGET_VIEW}_BEST_LOSS.pth"

pos_weight_val = 3.0
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight_val]).to(DEVICE))

model = NFLImpactModel(num_frames=16, hidden_dim=256, dropout_prob=0.5).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1.5e-5, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
scaler = torch.amp.GradScaler('cuda')


NUM_EPOCHS = 20
best_val_f1 = 0.0
best_val_loss = float('inf')

print(f"\n START TRAINING | View: {TARGET_VIEW} | Loss Weight: {pos_weight_val}x")

print(f" Validation Set: {len(val_pos_idxs)} Positives vs {num_neg_val} Negatives (Ratio 1:3)")

for epoch in range(NUM_EPOCHS):
    # --- TRAIN ---
    model.train()
    train_loss = 0.0
    all_train_preds, all_train_targets = [], []

    pbar = tqdm(train_loader, desc=f"Ep {epoch+1}/{NUM_EPOCHS} [Train]")
    for inputs, targets in pbar:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        preds_bin = (torch.sigmoid(outputs).max(dim=1)[0] > 0.5).detach().cpu().numpy()
        targets_bin = (targets.max(dim=1)[0] > 0.5).detach().cpu().numpy()
        all_train_preds.extend(preds_bin)
        all_train_targets.extend(targets_bin)

        pbar.set_postfix({'loss': f"{loss.item():.4f}"})

    avg_train_loss = train_loss / len(train_loader)
    train_f1 = f1_score(all_train_targets, all_train_preds, zero_division=0)


    # --- VALIDATION ---
    model.eval()
    val_loss = 0.0
    all_val_preds = []
    all_val_targets = []

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            preds_prob = torch.sigmoid(outputs)
            preds_bin = (preds_prob.max(dim=1)[0] > 0.45).cpu().numpy().astype(int)
            targets_bin = (targets.max(dim=1)[0] > 0.5).cpu().numpy().astype(int)

            all_val_preds.extend(preds_bin)
            all_val_targets.extend(targets_bin)

    avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0.0

    val_prec = precision_score(all_val_targets, all_val_preds, zero_division=0)
    val_rec = recall_score(all_val_targets, all_val_preds, zero_division=0)
    val_f1 = f1_score(all_val_targets, all_val_preds, zero_division=0)

    print(f"REPORT EPOCH {epoch+1}")
    print(f"   Train -> Loss: {avg_train_loss:.4f} | F1: {train_f1:.4f}")
    print(f"   Val   -> Loss: {avg_val_loss:.4f} | F1: {val_f1:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f}")

    scheduler.step(val_f1)

    # --- SALVATAGGIO ---
    if val_f1 > best_val_f1:
        print(f"   NEW BEST F1! ({best_val_f1:.4f} -> {val_f1:.4f}) Saving...")
        best_val_f1 = val_f1
        torch.save(model.state_dict(), path_best_f1)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), path_best_loss)
        print(f"   New Best Loss: {avg_val_loss:.4f}")

print(f"\nTraining complete.")
print(f"Model best F1: {path_best_f1}")

In [ ]:
files.download(path_best_f1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>